## Test gridded spatial interpolators using Franke bivariate test function  
http://www.sfu.ca/~ssurjano/franke2d.html  

In [ ]:
import numpy as np
import scipy.interpolate
import spatial_interpolators as spi
import matplotlib.pyplot as plt

### Franke's bivariate test function

In [ ]:
def franke(x,y):
	F1 = 0.75*np.exp(-((9.0*x-2.0)**2 + (9.0*y-2.0)**2)/4.0)
	F2 = 0.75*np.exp(-((9.0*x+1.0)**2/49.0-(9.0*y+1.0)/10.0))
	F3 = 0.5*np.exp(-((9.0*x-7.0)**2 + (9.0*y-3.0)**2)/4.0)
	F4 = 0.2*np.exp(-((9.0*x-4.0)**2 + (9.0*y-7.0)**2))
	F = F1 + F2 + F3 - F4
	return F

### Calculate Franke's evaluation function

In [ ]:
# calculate output points
nx = 250
ny = 250
xpts = np.arange(nx)/np.float64(nx)
ypts = np.arange(ny)/np.float64(ny)
XI,YI = np.meshgrid(xpts,ypts)
# calculate real values at grid points
ZI = np.ma.zeros((ny,nx))
ZI.mask = np.zeros((ny,nx),dtype=bool)
ZI.data[:] = franke(XI,YI)
# create random points to be removed from the grid
indx = np.random.randint(0, high=nx, size=32150)
indy = np.random.randint(0, high=ny, size=32150)
ZI.mask[indy,indx] = True

### Plot Original Franke Function with Missing Values

In [ ]:
# plot data and interpolated data
f1, ax1 = plt.subplots(num=1, figsize=(6,6))
extents=(0,1,1,0)

# create color map with invalid points
cmap = plt.cm.get_cmap('Spectral_r').copy()
cmap.set_bad('w',0.)
# plot read data with missing values
ax1.imshow(ZI, interpolation='nearest', extent=extents, cmap=cmap,
	vmin=ZI.min(), vmax=ZI.max())
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([]); ax1.get_yaxis().set_ticks([])
# set x and y limits
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
# add titles
ax1.set_title('Franke Function')
# subplot adjustments
f1.subplots_adjust(left=0.02,right=0.98,bottom=0.02,top=0.95,
	wspace=0.02,hspace=0.1)
plt.show()

### Interpolate missing values using inpainting algorithms

In [ ]:
interp = {}
interp['nearest'] = spi.inpaint(xpts, ypts, ZI, n=0)
interp['inpainted'] = spi.inpaint(xpts, ypts, ZI, n=100)

### Plot Interpolated Grids

In [ ]:
# plot data and interpolated data
f2,ax2 = plt.subplots(num=2, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# plot interpolated data with filled values
for i,key in enumerate(interp.keys()):
	ax2[i].imshow(interp[key],
        interpolation='nearest',
		extent=extents, cmap=cmap,
		vmin=ZI.min(), vmax=ZI.max())
	# no ticks on the x and y axes
	ax2[i].get_xaxis().set_ticks([])
	ax2[i].get_yaxis().set_ticks([])
	# set x and y limits
	ax2[i].set_xlim(0, 1)
	ax2[i].set_ylim(0, 1)
# add titles
ax2[0].set_title('Nearest Neighbors')
ax2[1].set_title('DCT Inpainted')
# subplot adjustments
f2.subplots_adjust(left=0.02,right=0.98,bottom=0.02,top=0.95,
	wspace=0.02,hspace=0.1)
plt.show()

In [ ]:
# calculate real values at grid points
ZAll = franke(XI,YI)
difference = {}
for key,val in interp.items():
	difference[key] = np.sqrt((ZAll - val)**2)

# plot data and interpolated data
f2,ax2 = plt.subplots(num=2, ncols=2, sharex=True, sharey=True, figsize=(12,6))

# create color map with invalid points
dmap = plt.cm.get_cmap('viridis').copy()
dmap.set_bad('w',0.)
# maximum value in differences
vmax = np.max([np.max(val) for key,val in difference.items()])
# inverse indices
invy,invx = np.nonzero(np.logical_not(ZI.mask))
ninv = np.count_nonzero(np.logical_not(ZI.mask))

# plot differences
for i,key in enumerate(difference.keys()):
	RMS = np.sqrt(np.sum(difference[key][indy,indx]**2)/ninv)
	print('{0} RMS: {1:0.6f}'.format(key,RMS))
	ax2[i].imshow(difference[key],
		interpolation='nearest',
		extent=extents, cmap=dmap,
		vmin=0, vmax=vmax)
	# no ticks on the x and y axes
	ax2[i].get_xaxis().set_ticks([])
	ax2[i].get_yaxis().set_ticks([])
	# set x and y limits
	ax2[i].set_xlim(0, 1)
	ax2[i].set_ylim(0, 1)
	# add titles
	ax2[0].set_title('Nearest Neighbors')
	ax2[1].set_title('DCT Inpainted')

# subplot adjustments
f2.subplots_adjust(left=0.02,right=0.98,bottom=0.02,top=0.95,
wspace=0.02,hspace=0.1)
plt.show()